<a href="https://colab.research.google.com/github/chekhd/projects/blob/main/%D0%90%D0%B2%D1%82%D0%BE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости.

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

**Цель проекта**: протестировать несколько моделей машинного обучения для прогнозирования рыночной цены автомобиля. Проанализировать полученные модели и выбрать более подходящую модель под критерии заказчика.

## Подготовка данных

### Импорт библиотек, загрузка данных

In [ ]:
!pip install pandas-profiling

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.0/353.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.7/102.7 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.5/679.5 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.4/455.4 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 24.8 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27081 sha256=047b8e7321eb666ee45857c54940bff8e8811e6acde50e8d7f6f8762b766e3e0
  Stored in directory: /root/.cache/pip/wheels/dd/91/29/a79cecb328d01739e64017b6fb9a1ab9d8cb1853098ec5966d
Successfully built htmlmin
  Attempting uninstall: wordcloud
    Found existing installation: wordcloud 1.8.2.2
    Uninstalling wordc

In [ ]:
# Импорт библиотек
import pandas as pd
import numpy as np

import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns

from statistics import mode as mode
import time

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.metrics import mean_squared_error
from sklearn.dummy import DummyRegressor

import lightgbm as lgb

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
from pandas_profiling import ProfileReport


<div class="alert alert-block alert-success">
<b>Успех:</b> Импорты  на месте
</div>


In [ ]:
# Загрузка данных
df = pd.read_csv('/datasets/autos.csv')

FileNotFoundError: ignored

In [ ]:
ProfileReport(df)

### Изучение данных

In [ ]:
print(f'Количество строк: {df.shape[0]}, количество столбцов: {df.shape[1]}')

In [ ]:
df.info()

In [ ]:
display(df.head(3))

Наблюдаем, что в датасете 354369 строк, 16 столбцов. Названия столбцов не приведены к нижнему регистру, в некоторых столбцах имеются пропуски, типы данных соответствуют представленным данным.

In [ ]:
df.hist(figsize=(12,8))
plt.show()

Возможны аномалии в столбцах с годом регистрации и мощностью.

### Предобработка данных

#### Приведение названия столбцов к нижнему регистру

In [ ]:
df.columns = df.columns.str.lower()

In [ ]:
# проверка названий колонок
df.columns

#### Работа с пропусками, работа с выбросами

In [ ]:
print(df.isna().sum().sort_values(ascending=False))

Наблюдаем наличие пропусков в пяти признаках: repaired, vehicletype, fueltype, gearbox, model

In [ ]:
# столбец repaired
df['repaired'].unique()

Проверим какая доля автомобилей, зарегистрированных до 2010 года была в ремонте. После этого заполним пропуски в столбце repaired

In [ ]:
print(np.array(sorted(df['registrationyear'].unique())))

Наблюдаем, что в столбце registrationyear присутствуют аномалии. Так, минимальный год регистрации автомобиля - 1000, а максимальный = 9999. Признаём аномалиями все данные выше 2019 года. А для определения нижнего порога построим боксплот

In [ ]:
fig, ax = plt.subplots(figsize=(8,5), dpi=100)
sns.boxplot(df['registrationyear'])
plt.gca().set_xlim([1900, 2020])
plt.show()
df['registrationyear'].describe()

Можно понять, что "нижний ус" начинается на отметке 1985. Остальное признаём выбросами и удаляем

In [ ]:
# Посчитаем долю выбросов в столбце registrationyear
print((df.loc[(df['registrationyear']<1985) | (df['registrationyear']>2019)]['registrationyear'].count()/df.shape[0])*100)
# Выбросы составляют 1,57%, поэтому удаляем

In [ ]:
# Удаляем аномалии в столбце registrationyear
df = df.loc[(df['registrationyear']>1984) & (df['registrationyear']<2020)]
# Проверяем значения
print(np.array(sorted(df['registrationyear'].unique())))

In [ ]:
(pd.to_datetime(df['datecrawled'])).describe()

In [ ]:
# Посчитаем долю машин, которые были в ремонте
print(f'Доля машин, побывавших в ремонте: {round(((df.loc[df["repaired"]=="yes"]["repaired"].count())/df.shape[0])*100, 2)}')
print()

repaired_by_year = df.loc[df["repaired"]=="yes"].value_counts(["registrationyear"], sort = False)
all_cars_by_year = (df.value_counts('registrationyear', sort = False))
share_repaired_by_year = pd.Series(repaired_by_year.values/all_cars_by_year.values, index = all_cars_by_year.index)

In [ ]:
# Распределение ромонтируемых машин во времени
share_repaired_by_year.plot(kind = 'bar', figsize = (11,6))
plt.title('Доля авто побывавших в ремонте с распределениев по годам регистрации')
plt.show()
share_repaired_by_year.describe()


Можем наблюдать, что кол-во автомобилей зарегистрированных в 1986-1987, 1989-2002 и в 2016гг. были в ремонте гораздо чаще среднего значения. Так, например, в 1997 году доля авто, которые были в ремонте составила 16%, в то время как среднее значение показателя около 10%. Так, понимаем, что в 1997 году показатель превышает среднее значение на 60%.

Тем не менее, мне не представляется возможным заполнить пропуски в столбце repaired в зависимость от года регистрации, поэтому, заполним их заглушкой unknown

In [ ]:
# заполнение пропусков в столбце repaired
df['repaired'] = df['repaired'].fillna('unknown')
# проверим заполнение
print(f'Количество пропусков: {df["repaired"].isna().sum()}')
print(df['repaired'].unique())

In [ ]:
# столбец vehicletype
df['vehicletype'].unique()

Заполним пропуски в столбце vehicletype по самым распространенным типам кузова модели

In [ ]:
# Проверим, если есть пропуски в названиях модели в тех объектах, где есть пропуски в типе кузова
df.loc[(df['vehicletype'].isna()) & (df['model'].isna())]['registrationyear'].count()
# Да, присутвтвет 6471 объект, поэтому сначала заполним пропуски в столбце model

Заполним модель автомобиля с помощью марки и мощности.  
Проверим на нескольких брендах, будет ли существенно отличаться среднняя мощность автомобиля в зависимость от модели.

In [ ]:
# создадим функцию
df.loc[df['brand']=='volkswagen']['power']
def model_power(brand):
    model_list = []
    power_list = []
    for model in (df.loc[df['brand']== brand]['model']).unique():
        model_list.append(model)
        power_list.append(df.loc[df['model']==model]['power'].mean())
    model_power = (pd.Series(index=model_list, data=power_list))
    model_power.plot(kind = 'bar', figsize = (10,5))
    plt.title(f'Средняя мощность авто в зависимости от модели. {brand}')
    plt.show()
    print(model_power.describe())

In [ ]:
model_power('volkswagen')
model_power('audi')

На примере volkswagen и audi можем наблюдать, что разница между некоторыми моделями существенна.  
Выделим три категории мощности для каждого бренда и внутри каждой категории модой выберем модель авто.  
Но перед этим рассмотрим столбец с мощностями на аномалии.

In [ ]:
for brand in df['brand'].unique():
    fig, ax = plt.subplots(figsize=(8,5), dpi=100)
    sns.boxplot(df.loc[df['brand']==brand]['power'])
    plt.gca().set_xlim([0, 1500])
    plt.title(f'Распределение мощностей. {brand}')
    plt.show()
    df['power'].describe()

Наблюдаем очень большое кол-во значений, которые во много раз превосходят медиану.  
Предполагаем, что в выборке могут быть представлены объекты в спортивной сборке, поэтому нельзя просто удалить все объекты, которые находятся на числовой оси дальше "правого уса". По данной причине принято решение расширить область допустимых значений с 1,5 до 3-х межквартильных размахов.


<div class="alert alert-block alert-info">
<b>Совет:</b> Околонулевые цены и мощности не кажутся подозрительными?

In [ ]:
# Удалим выбросы в столбце power:
for brand in df['brand'].unique():
    q1, q3 = np.percentile(df.loc[df['brand']==brand]['power'], [25, 75])
    iqr = q3 - q1
    upper_bound = q3 + 3 * iqr
    df.loc[(df['brand']==brand)] = df.loc[(df['brand']==brand) & (df['power']<upper_bound)]

In [ ]:
# Удалим пропуски в графе бренд
df = df.loc[~df['brand'].isna()]

In [ ]:
# Замечаем также, что у sonstige_autos вообще отсутствуют названия моделей. Поэтому заполним их sonstige_autos
df.loc[(df['brand']=='sonstige_autos') & (df['model'].isna()), 'model'] = 'sonstige_autos'

Заполним модель авто:

In [ ]:
for brand in df['brand'].unique():
    q1, q3 = np.percentile(df.loc[df['brand']==brand]['power'], [25, 75])

    df.loc[(
            df['brand']==brand) & (df['model'].isna()) & (df['power']<=q1), 'model'] = df.loc[(
            df['brand']==brand) & (df['power']<=q1), 'model'].mode()[0]
    df.loc[(
            df['brand']==brand) & (df['model'].isna()) & (df['power']<=q3) & (df['power']>q1), 'model'] = df.loc[(
            df['brand']==brand) & (df['power']<=q3) & (df['power']>q1), 'model'].mode()[0]
    df.loc[(
            df['brand']==brand) & (df['model'].isna()) & (df['power']>q3), 'model'] = df.loc[(
            df['brand']==brand) & (df['power']>q3), 'model'].mode()[0]


Заполним пропуски в столбце vehicletype по самым распространенным типам кузова модели

In [ ]:
df.loc[df['model']=='serie_2', 'vehicletype'] = 'unknown'

In [ ]:
for model in df['model'].unique():
    df.loc[(
        df['model']==model) & (df['vehicletype'].isna()), 'vehicletype'] = df.loc[(
        df['model']==model), 'vehicletype'].mode()[0]

In [ ]:
# Проверим наличие пропусков в типе кузова:
df['vehicletype'].isna().sum()

In [ ]:
# Заполним пропуски в столбцах fueltype и gearbox формулировкой unknown
df.loc[df['fueltype'].isna(), 'fueltype'] = 'unknown'
df.loc[df['gearbox'].isna(), 'gearbox'] = 'unknown'

In [ ]:
# Проверим наличие пропусков
print(df.isna().sum().sort_values(ascending=False))

#### Проверка на наличие явных дубликатов

In [ ]:
df.loc[df.duplicated()]

In [ ]:
df = df.drop_duplicates().reset_index(drop=True)

In [ ]:
df.duplicated().sum()

Явные дубликаты удалены

#### Промежуточные выводы

В данном пункте была выполнена работа по предобработке данных. Названия колонок были приведены к нижнему регистру, были заполнены пропуски и удалены выбросы и аномалии, а также выполнен поиск явных дубликатов.



## Обучение моделей

### Удаление неинформативных признаков

In [ ]:
df.head(3)

,datecrawled,price,vehicletype,registrationyear,gearbox,power,model,kilometer,registrationmonth,fueltype,brand,repaired,datecreated,numberofpictures,postalcode,lastseen
0,2016-03-24 11:52:17,480.0,sedan,1993.0,manual,0.0,golf,150000.0,0.0,petrol,volkswagen,unknown,2016-03-24 00:00:00,0.0,70435.0,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300.0,coupe,2011.0,manual,190.0,a6,125000.0,5.0,gasoline,audi,yes,2016-03-24 00:00:00,0.0,66954.0,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800.0,suv,2004.0,auto,163.0,grand,125000.0,8.0,gasoline,jeep,unknown,2016-03-14 00:00:00,0.0,90480.0,2016-04-05 12:47:46


In [ ]:
# Удалим столбцы, которые кажутся неинформативными
drop_columns = ['datecrawled', 'registrationmonth', 'datecreated', 'postalcode', 'lastseen', 'numberofpictures']
df_droped_columns = df_drop_columns = df.copy().drop(columns=drop_columns)

In [ ]:
df_droped_columns.head()

,price,vehicletype,registrationyear,gearbox,power,model,kilometer,fueltype,brand,repaired
0,480.0,sedan,1993.0,manual,0.0,golf,150000.0,petrol,volkswagen,unknown
1,18300.0,coupe,2011.0,manual,190.0,a6,125000.0,gasoline,audi,yes
2,9800.0,suv,2004.0,auto,163.0,grand,125000.0,gasoline,jeep,unknown
3,1500.0,small,2001.0,manual,75.0,golf,150000.0,petrol,volkswagen,no
4,3600.0,small,2008.0,manual,69.0,fabia,90000.0,gasoline,skoda,no


### Разделим выборки

In [ ]:
target = df_droped_columns['price']
features = df_droped_columns.drop(['price'], axis = 1)

In [ ]:
features_train, features_valid_test, target_train, target_valid_test = train_test_split(
    features, target, test_size=0.4, random_state=12345)
features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid_test, target_valid_test, test_size=0.5, random_state=12345)

In [ ]:
def shape_(df):
    print(df.shape)
# Проверим кол-во строк и столбцов в выборках
shape_(features_train)
shape_(target_train)
shape_(features_valid)
shape_(target_valid)
shape_(features_test)
shape_(target_test)

(208501, 9)
(208501,)
(69501, 9)
(69501,)
(69501, 9)
(69501,)


### Дамми модель

In [ ]:
dummy_model = DummyRegressor(strategy="mean")
# Обучаем константную модель
dummy_model.fit(features_train, target_train)

DummyRegressor()

In [ ]:
predictions_valid = dummy_model.predict(features_valid)
result = np.sqrt(mean_squared_error(target_valid, predictions_valid))
print(f'RMSE дамми модели на валидационной выборке = {result}')

RMSE дамми модели на валидационной выборке = 4466.630639591759


Результат дамми модели 4466.63 на валидационной выборке.

### Линейная регрессия

#### Кодируем признаки для линейной регрессии

In [ ]:
# Кодируем признаки для линейной регрессии

features_categirical = ['vehicletype', 'gearbox', 'model', 'fueltype', 'brand', 'repaired']

# кодируем выборки
ohe = OneHotEncoder(sparse=False, drop='first')
ohe.fit(features_train[features_categirical])

def features_ohe(ohe_variable, df_features, features_categ):
    df_features_ohe = pd.DataFrame(
        data=ohe_variable.transform(df_features[features_categ]),
        index=df_features.index,
        columns=ohe_variable.get_feature_names()
    )

    df_features = df_features.drop(features_categ, axis=1)
    df_features = df_features.join(df_features_ohe)
    return df_features

features_train_OHE = features_ohe(ohe, features_train, features_categirical);
features_valid_OHE = features_ohe(ohe, features_valid, features_categirical);
features_test_OHE = features_ohe(ohe, features_test, features_categirical);

In [ ]:
# Проверим выборки после кодирования
shape_(features_train_OHE)
shape_(features_valid_OHE)
shape_(features_test_OHE)

(208501, 309)
(69501, 309)
(69501, 309)


Ожидаемо, получили очень большое количество признаков, но т.к. мы применили OHE для линейной модели, это не должно сильно отразиться на времени обучения

#### Масштабируем признаки для линейной модели

In [ ]:
# масштабируем признаки
scaler = StandardScaler()
scaler.fit(features_train_OHE)

features_train_OHE_scaled = pd.DataFrame(
    scaler.transform(features_train_OHE), columns=features_train_OHE.columns, index = features_train_OHE.index)
features_valid_OHE_scaled = pd.DataFrame(
    scaler.transform(features_valid_OHE), columns=features_valid_OHE.columns, index = features_valid_OHE.index)
features_test_OHE_scaled = pd.DataFrame(
    scaler.transform(features_test_OHE), columns=features_test_OHE.columns, index = features_test_OHE.index)

#### Обучение линейной модели

In [ ]:
model_lr = LinearRegression()
start_time = time.time()
model_lr.fit(features_train_OHE_scaled, target_train)
end_time = time.time()
print("Training time: {} seconds".format(end_time - start_time))

Training time: 26.653035640716553 seconds


In [ ]:
start_time = time.time()
predictions_valid = model_lr.predict(features_valid_OHE_scaled)
end_time = time.time()
print("Prediction time: {} seconds".format(end_time - start_time))
result = np.sqrt(mean_squared_error(target_valid, predictions_valid))
print(f'RMSE линейной регресии на валидационной выборке = {result}')

Prediction time: 0.10604095458984375 seconds
RMSE линейной регресии на валидационной выборке = 2674.1641299901066


Модель линейной регрессии демонстрирует результат, превосходящий дамми модель, но при этом, хуже, чем установленный порог RMSE в 2500. По данной причине на тестовой выборке линейная регрессия использоваться не будет

### LightGBM

#### Порядковое кодирование для lgb

In [ ]:
encoder = OrdinalEncoder(handle_unknown='ignore')
# categorical_features = ['vehicletype', 'gearbox', 'model', 'fueltype', 'brand', 'repaired']
encoder.fit(features_train)
features_train_encoded = pd.DataFrame(encoder.transform(features_train), columns=features_train.columns)
features_valid_encoded = pd.DataFrame(encoder.transform(features_valid), columns=features_valid.columns)
features_test_encoded = pd.DataFrame(encoder.transform(features_test), columns=features_test.columns)


#### Обучение модели

In [ ]:
# Создание объекта для обучения
train_data = lgb.Dataset(data=features_train_encoded, label=target_train)

# Определение параметров модели
params = {
    "objective": "regression",
    "metric": "rmse",
    "num_leaves": 5,
    "learning_rate": 0.1
}

valid_set = lgb.Dataset(data=features_valid_encoded, label=target_valid)

In [ ]:
# Обучение модели
start_time = time.time()

model_bust = lgb.train(
    params=params,
    train_set=train_data,
    num_boost_round=1000,
    valid_sets=train_data,
    early_stopping_rounds=50,
    verbose_eval=50
)

end_time = time.time()
print("Training time: {} seconds".format(end_time - start_time))

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.309497 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 569
[LightGBM] [Info] Number of data points in the train set: 208501, number of used features: 9
[LightGBM] [Info] Start training from score 4377.209136
Training until validation scores don't improve for 50 rounds
[50]	training's rmse: 2159.13
[100]	training's rmse: 2018.51
[150]	training's rmse: 1964.45
[200]	training's rmse: 1925.17
[250]	training's rmse: 1902.25
[300]	training's rmse: 1882.88
[350]	training's rmse: 1865.67
[400]	training's rmse: 1852.11
[450]	training's rmse: 1838.27
[500]	training's rmse: 1827.49
[550]	training's rmse: 1818.13
[600]	training's rmse: 1810.5
[650]	training's rmse: 1802.25
[700]	training's rmse: 1794.65
[750]	training's rmse: 1787.43
[800]	training's rmse: 1780.4
[850]	training's rmse: 1773.98
[900

In [ ]:
start_time = time.time()
prediction = model_bust.predict(features_valid_encoded)
end_time = time.time()
print("Prediction time: {} seconds".format(end_time - start_time))

result = np.sqrt(mean_squared_error(target_valid, prediction))
print(f'RMSE градиентного бустинга на валидационной выборке = {result}')

Наблюдаем, что LightGBM демонстрирует лучшие результаты относительно модели линейной регрессии. Так, на валидационной выборке наблюдаем значения RMSE = 1771.99, что значетельно ниже установленного порога в 2500, поэтому модель LightGBM проверим на тестовой выборке.

In [ ]:
start_time = time.time()
prediction_test = model_bust.predict(features_test_encoded)
end_time = time.time()
print("Prediction time: {} seconds".format(end_time - start_time))

result = np.sqrt(mean_squared_error(target_test, prediction_test))
print(f'RMSE градиентного бустинга на тестовой выборке = {result}')

## Анализ моделей

В данном проекте была проделана работа по построению модели для прогноза рыночной стоимости автомобилей.  
Было протестировано два вида моделей: линейной регрессии и модель градиентного бустинга LightGBM.

Модель линейной регрессии обучалась в течении 25 секунд и выдавала предсказания за 0,02 секунды, однако точность прогноза на валидационной выборке оказалась очень низкой: RMSE = 2674.  
Модель LightGB обучалась в течении 170 секунд и выдавала предсказания за 2.7-2.9 секунды, что конечно, дольше линейной модели, но точность предсказания намного выше: на валидационной выборке RMSE = 1772, а на тестовой выборке = 1799.48.

На основании данных фактов, можно сделать вывод, что для разработки приложения лучше подойдет модель гражиентного бустинга LightGB.